In [31]:
# Quick fix for missing packages - run this first
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ Installed {package}")
    except Exception as e:
        print(f"❌ Failed to install {package}: {e}")

# Install required packages
packages = [
    "flask", 
    "requests", 
    "python-dotenv", 
    "langchain-openai", 
    "langgraph", 
    "langchain-core",
    "twilio"
]

for pkg in packages:
    install_package(pkg)


✅ Installed flask
✅ Installed requests
✅ Installed python-dotenv
✅ Installed langchain-openai
✅ Installed langgraph
✅ Installed langchain-core
✅ Installed twilio


In [32]:
# Minimal test server to check if Flask works
from flask import Flask, request, jsonify

def test_minimal_server():
    """Test if Flask can start at all"""
    try:
        app = Flask(__name__)
        
        @app.route("/test")
        def test():
            return {"status": "Flask is working!"}
        
        print("✅ Flask app created successfully")
        print("🚀 Starting minimal server on port 5001...")
        app.run(host="0.0.0.0", port=5001, debug=False)
        
    except Exception as e:
        print(f"❌ Flask startup failed: {e}")
        return False

# Run this to test if Flask works
print("Testing minimal Flask server...")


Testing minimal Flask server...


In [33]:
# Check environment setup
import os
from dotenv import load_dotenv

def check_environment():
    """Check if environment variables are set up correctly"""
    load_dotenv()
    
    required_vars = [
        "TRAVELPORT_APPLICATION_KEY",
        "TRAVELPORT_APPLICATION_SECRET", 
        "TRAVELPORT_USERNAME",
        "TRAVELPORT_PASSWORD",
        "TRAVELPORT_ACCESS_GROUP"
    ]
    
    optional_vars = [
        "OPENAI_API_KEY",
        "WEBHOOK_VERIFY_TOKEN"
    ]
    
    print("🔍 Checking environment variables...")
    
    missing_required = []
    for var in required_vars:
        if os.getenv(var):
            print(f"✅ {var}: Set")
        else:
            print(f"❌ {var}: Missing")
            missing_required.append(var)
    
    for var in optional_vars:
        if os.getenv(var):
            print(f"✅ {var}: Set")
        else:
            print(f"⚠️  {var}: Missing (optional but recommended)")
    
    if missing_required:
        print(f"\n❌ Missing required variables: {missing_required}")
        print("Create a .env file with these variables")
        return False
    else:
        print("\n✅ All required environment variables are set!")
        return True

# Run the check
check_environment()


🔍 Checking environment variables...
✅ TRAVELPORT_APPLICATION_KEY: Set
✅ TRAVELPORT_APPLICATION_SECRET: Set
✅ TRAVELPORT_USERNAME: Set
✅ TRAVELPORT_PASSWORD: Set
✅ TRAVELPORT_ACCESS_GROUP: Set
✅ OPENAI_API_KEY: Set
✅ WEBHOOK_VERIFY_TOKEN: Set

✅ All required environment variables are set!


True

In [34]:
# Fixed server function - disable reloader for Jupyter compatibility
def run_server_fixed():
    """Run the Flask server with Jupyter-compatible settings"""
    print("🚀 Starting WhatsApp Flight Booking Bot...")
    print("💡 Make sure to set up your environment variables first!")
    
    try:
        # Import the full chatbot code here or reference your existing app
        from flask import Flask
        app = Flask(__name__)
        
        @app.route("/health")
        def health():
            return {"status": "healthy", "service": "Flight Booking Bot"}
        
        @app.route("/webhook", methods=["GET", "POST"])  
        def webhook():
            return "Webhook endpoint ready"
        
        print("✅ Flask app configured")
        print("🌐 Server will be available at:")
        print("   - http://localhost:5000/health")
        print("   - http://localhost:5000/webhook")
        
        # Fixed: disable reloader for Jupyter compatibility
        app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)
        
    except Exception as e:
        print(f"❌ Server startup failed: {e}")

print("✅ Fixed server function ready - run with: run_server_fixed()")


✅ Fixed server function ready - run with: run_server_fixed()


In [35]:
# Install required packages
%pip install langgraph langchain-openai langchain-core python-dotenv twilio flask requests dateparser


Note: you may need to restart the kernel to use updated packages.


In [36]:
import os
import json
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv
from typing import Dict, List, Optional, TypedDict
import dateparser
import re

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# WhatsApp/Twilio imports
from flask import Flask, request, jsonify
from twilio.twiml.messaging_response import MessagingResponse

# Load environment variables
load_dotenv()

# Travelport API Configuration
CLIENT_ID = os.getenv("TRAVELPORT_APPLICATION_KEY")
CLIENT_SECRET = os.getenv("TRAVELPORT_APPLICATION_SECRET")
USERNAME = os.getenv("TRAVELPORT_USERNAME")
PASSWORD = os.getenv("TRAVELPORT_PASSWORD")
ACCESS_GROUP = os.getenv("TRAVELPORT_ACCESS_GROUP")

OAUTH_URL = "https://oauth.pp.travelport.com/oauth/oauth20/token"
CATALOG_URL = "https://api.pp.travelport.com/11/air/catalog/search/catalogproductofferings"

# OpenAI Configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("✅ Configuration loaded")


✅ Configuration loaded


In [37]:
# Travelport API Authentication
def fetch_password_token():
    """Get OAuth token from Travelport API"""
    data = {
        "grant_type": "password",
        "username": USERNAME,
        "password": PASSWORD,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "openid"
    }
    resp = requests.post(
        OAUTH_URL,
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data=data
    )
    resp.raise_for_status()
    return resp.json()["access_token"]

def get_api_headers():
    """Build headers for Travelport API requests"""
    token = fetch_password_token()
    return {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Accept-Encoding": "gzip, deflate",
        "Cache-Control": "no-cache",
        "Authorization": f"Bearer {token}",
        "XAUTH_TRAVELPORT_ACCESSGROUP": ACCESS_GROUP,
        "Accept-Version": "11",
        "Content-Version": "11",
    }

print("✅ Travelport API functions ready")


✅ Travelport API functions ready


In [38]:
# LangGraph State Definition
class FlightBookingState(TypedDict):
    messages: List[HumanMessage | AIMessage]
    user_message: str
    from_city: Optional[str]
    to_city: Optional[str]
    departure_date: Optional[str]
    return_date: Optional[str]
    passengers: int
    passenger_age: int
    raw_api_response: Optional[Dict]
    cheapest_flight: Optional[Dict]
    response_text: str

# Initialize LLM (API key will be picked up from OPENAI_API_KEY env var)
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

print("✅ LangGraph state and LLM initialized")


✅ LangGraph state and LLM initialized


In [39]:
# Message Parsing Function
def parse_travel_request(state: FlightBookingState) -> FlightBookingState:
    """Parse user message to extract travel details using LLM"""
    
    parsing_prompt = f"""
    Extract flight booking details from this user message: "{state['user_message']}"
    
    Return ONLY a JSON object with these fields:
    {{
        "from_city": "3-letter airport code or null",
        "to_city": "3-letter airport code or null", 
        "departure_date": "YYYY-MM-DD format or null",
        "return_date": "YYYY-MM-DD format or null",
        "passengers": "number of passengers (default 1)",
        "passenger_age": "age of passenger (default 25)"
    }}
    
    Rules:
    - Use standard 3-letter IATA airport codes (e.g., LHE for Lahore, ATH for Athens)
    - Parse relative dates like "next week", "in 30 days" into actual dates
    - If return_date is not mentioned but it seems like a round trip, set it to 7 days after departure
    - If information is missing, use null
    
    Example: "I want to fly from Lahore to Athens next month and come back after a week"
    Output: {{"from_city": "LHE", "to_city": "ATH", "departure_date": "2024-02-15", "return_date": "2024-02-22", "passengers": 1, "passenger_age": 25}}
    """
    
    try:
        response = llm.invoke([HumanMessage(content=parsing_prompt)])
        content = response.content if isinstance(response.content, str) else str(response.content)
        parsed_data = json.loads(content)
        
        # Update state with parsed information
        state.update({
            "from_city": parsed_data.get("from_city"),
            "to_city": parsed_data.get("to_city"),
            "departure_date": parsed_data.get("departure_date"),
            "return_date": parsed_data.get("return_date"),
            "passengers": parsed_data.get("passengers", 1),
            "passenger_age": parsed_data.get("passenger_age", 25)
        })
        
        print(f"✅ Parsed travel details: {parsed_data}")
        
    except Exception as e:
        print(f"❌ Error parsing message: {e}")
        state["response_text"] = "I couldn't understand your flight request. Please provide details like: from city, to city, and travel dates."
    
    return state

print("✅ Message parsing function ready")


✅ Message parsing function ready


In [40]:
headers = get_api_headers()


departure_date = (datetime.utcnow() + timedelta(days=30)).strftime("%Y-%m-%d")
return_date    = (datetime.utcnow() + timedelta(days=37)).strftime("%Y-%m-%d")
from_city = "LHE"
to_city = "ATH"
carrier_list = ["QR", "EY", "GF", "SV"]

payload = {
    "@type": "CatalogProductOfferingsQueryRequest",
    "CatalogProductOfferingsRequest": {
        "@type": "CatalogProductOfferingsRequestAir",
        "maxNumberOfUpsellsToReturn": 1,
        "contentSourceList": ["GDS"],
        "PassengerCriteria": [
            {
                "@type": "PassengerCriteria",
                "number": 1,
                "age": 25,
                "passengerTypeCode": "ADT"
            }
        ],
        "SearchCriteriaFlight": [
            {
                "@type": "SearchCriteriaFlight",
                "departureDate": departure_date,   
                "From": {"value": from_city},
                "To":   {"value": to_city}
            },
            {
                "@type": "SearchCriteriaFlight",
                "departureDate": return_date,      
                "From": {"value": to_city},
                "To":   {"value": from_city}
            }
        ],
        "SearchModifiersAir": {
            "@type": "SearchModifiersAir",
            "CarrierPreference": [
                {
                    "@type": "CarrierPreference",
                    "preferenceType": "Preferred",
                    "carriers": carrier_list
                }
            ]
        },
        "CustomResponseModifiersAir": {
            "@type": "CustomResponseModifiersAir",
            "SearchRepresentation": "Journey"
        }
    }
}

In [41]:
response = requests.post(CATALOG_URL, headers=headers, json=payload)
try:
    response.raise_for_status()
    result = response.json()
    import json
    print(json.dumps(result, indent=2))
except requests.HTTPError as e:
    print("❌ Request failed:", e)
    print(response.status_code, response.text)

{
  "CatalogProductOfferingsResponse": {
    "@type": "CatalogProductOfferingsResponse",
    "transactionId": "f735ff6d-2208-42ba-a9f8-4651a7145946",
    "CatalogProductOfferings": {
      "@type": "CatalogProductOfferings",
      "CatalogProductOffering": [
        {
          "@type": "CatalogProductOffering",
          "sequence": 1,
          "id": "o1",
          "Departure": "LHE",
          "Arrival": "ATH",
          "Brand": [
            {
              "@type": "BrandID",
              "BrandRef": "b0"
            },
            {
              "@type": "BrandID",
              "BrandRef": "b1"
            }
          ],
          "ProductBrandOptions": [
            {
              "@type": "ProductBrandOptions",
              "flightRefs": [
                "s17",
                "s18"
              ],
              "ProductBrandOffering": [
                {
                  "@type": "ProductBrandOffering",
                  "Brand": {
                    "@type": "Brand

In [42]:
# Flight Search Function
def search_flights(state: FlightBookingState) -> FlightBookingState:
    """Search for flights using Travelport API"""
    
    # Check if we have required information
    if not state.get("from_city") or not state.get("to_city") or not state.get("departure_date"):
        state["response_text"] = "I need more information. Please specify: departure city, destination city, and travel date."
        return state
    
    try:
        # Build search criteria
        search_criteria = [{
            "@type": "SearchCriteriaFlight",
            "departureDate": state["departure_date"],
            "From": {"value": state["from_city"]},
            "To": {"value": state["to_city"]}
        }]
        
        # Add return flight if specified
        if state.get("return_date"):
            search_criteria.append({
                "@type": "SearchCriteriaFlight", 
                "departureDate": state["return_date"],
                "From": {"value": state["to_city"]},
                "To": {"value": state["from_city"]}
            })
        
        # Build API payload
        payload = {
            "@type": "CatalogProductOfferingsQueryRequest",
            "CatalogProductOfferingsRequest": {
                "@type": "CatalogProductOfferingsRequestAir",
                "maxNumberOfUpsellsToReturn": 1,
                "contentSourceList": ["GDS"],
                "PassengerCriteria": [{
                    "@type": "PassengerCriteria",
                    "number": state.get("passengers", 1),
                    "age": state.get("passenger_age", 25),
                    "passengerTypeCode": "ADT"
                }],
                "SearchCriteriaFlight": search_criteria,
                "SearchModifiersAir": {
                    "@type": "SearchModifiersAir",
                    "CarrierPreference": [{
                        "@type": "CarrierPreference",
                        "preferenceType": "Preferred",
                        "carriers": ["QR", "EY", "GF", "SV"]  # Preferred carriers
                    }]
                },
                "CustomResponseModifiersAir": {
                    "@type": "CustomResponseModifiersAir",
                    "SearchRepresentation": "Journey"
                }
            }
        }
        
        # Make API call
        headers = get_api_headers()
        response = requests.post(CATALOG_URL, headers=headers, json=payload)
        response.raise_for_status()
        
        api_result = response.json()
        state["raw_api_response"] = api_result
        
        print(f"✅ Flight search completed. Found {len(api_result.get('CatalogProductOfferingsResponse', {}).get('CatalogProductOfferings', []))} options")
        
    except Exception as e:
        print(f"❌ Flight search error: {e}")
        state["response_text"] = f"Sorry, I couldn't search for flights at the moment. Error: {str(e)}"
    
    return state

print("✅ Flight search function ready")


✅ Flight search function ready


In [43]:
# Flight Analysis Function
def find_cheapest_flight(state: FlightBookingState) -> FlightBookingState:
    """Analyze API response to find cheapest flight and extract details"""
    
    if not state.get("raw_api_response"):
        state["response_text"] = "No flight data available to analyze."
        return state
    
    try:
        api_response = state["raw_api_response"]
        response_data = api_response.get("CatalogProductOfferingsResponse", {}) if api_response else {}
        offerings = response_data.get("CatalogProductOfferings", []) if isinstance(response_data, dict) else []
        
        if not offerings:
            state["response_text"] = "No flights found for your search criteria."
            return state
        
        cheapest_flight = None
        lowest_price = float('inf')
        
        # Find the cheapest flight
        for offering in offerings:
            try:
                # Extract price information
                price_info = offering.get("TotalPrice", {})
                if isinstance(price_info, list) and price_info:
                    price_info = price_info[0]
                
                total_value = price_info.get("Total", {}) if isinstance(price_info, dict) else {}
                total_price = float(total_value.get("value", 0) if isinstance(total_value, dict) else 0)
                
                if total_price > 0 and total_price < lowest_price:
                    lowest_price = total_price
                    cheapest_flight = offering
                    
            except (ValueError, TypeError, KeyError) as e:
                continue
        
        if cheapest_flight:
            state["cheapest_flight"] = cheapest_flight
            
            # Extract flight details
            flight_details = extract_flight_details(cheapest_flight)
            state["response_text"] = format_flight_response(flight_details)
            
            print(f"✅ Found cheapest flight: ${lowest_price}")
        else:
            state["response_text"] = "I found flights but couldn't determine pricing. Please try again."
            
    except Exception as e:
        print(f"❌ Error analyzing flights: {e}")
        state["response_text"] = "Sorry, I had trouble analyzing the flight options."
    
    return state

def extract_flight_details(flight_offering: Dict) -> Dict:
    """Extract relevant details from a flight offering"""
    details = {
        "price": "N/A",
        "currency": "USD",
        "departure_time": "N/A",
        "arrival_time": "N/A", 
        "duration": "N/A",
        "airline": "N/A",
        "baggage": "Check with airline",
        "stops": "N/A"
    }
    
    try:
        # Extract price
        price_info = flight_offering.get("TotalPrice", {})
        if isinstance(price_info, list) and price_info:
            price_info = price_info[0]
        
        total_info = price_info.get("Total", {}) if isinstance(price_info, dict) else {}
        details["price"] = total_info.get("value", "N/A") if isinstance(total_info, dict) else "N/A"
        details["currency"] = total_info.get("currencyCode", "USD") if isinstance(total_info, dict) else "USD"
        
        # Extract journey details
        journeys = flight_offering.get("Product", [])
        if journeys and isinstance(journeys, list):
            journey = journeys[0]
            
            # Get flight segments
            segments = journey.get("JourneyDetail", {}).get("FlightSegment", [])
            if segments:
                first_segment = segments[0] if isinstance(segments, list) else segments
                last_segment = segments[-1] if isinstance(segments, list) and len(segments) > 1 else first_segment
                
                # Departure and arrival times
                details["departure_time"] = first_segment.get("DepartureDateTime", "N/A")
                details["arrival_time"] = last_segment.get("ArrivalDateTime", "N/A")
                
                # Airline
                details["airline"] = first_segment.get("MarketingCarrier", {}).get("Name", "N/A")
                
                # Number of stops
                details["stops"] = "Direct" if len(segments) == 1 else f"{len(segments) - 1} stop(s)"
        
        # Extract baggage information
        baggage_info = flight_offering.get("Product", [{}])[0].get("BaggageAllowance", [])
        if baggage_info:
            details["baggage"] = f"{baggage_info[0].get('MaximumWeight', {}).get('value', 'Standard')} {baggage_info[0].get('MaximumWeight', {}).get('unit', 'kg')}"
            
    except Exception as e:
        print(f"Warning: Could not extract some flight details: {e}")
    
    return details

def format_flight_response(details: Dict) -> str:
    """Format flight details into a user-friendly response"""
    
    return f"""✈️ *Flight Found!*

💰 *Price:* {details['currency']} {details['price']}
🛫 *Departure:* {details['departure_time']}
🛬 *Arrival:* {details['arrival_time']}
✈️ *Airline:* {details['airline']}
🔄 *Stops:* {details['stops']}
🧳 *Baggage:* {details['baggage']}

Would you like me to search for more options or help you with booking?"""

print("✅ Flight analysis functions ready")


✅ Flight analysis functions ready


In [44]:
# LangGraph Workflow Definition
def should_search_flights(state: FlightBookingState) -> str:
    """Decision function to determine next step"""
    if state.get("response_text") and "couldn't understand" in state["response_text"]:
        return "end"
    if not state.get("from_city") or not state.get("to_city") or not state.get("departure_date"):
        return "end"
    return "search"

def should_analyze_flights(state: FlightBookingState) -> str:
    """Decision function after flight search"""
    if state.get("raw_api_response"):
        return "analyze"
    return "end"

# Create the LangGraph workflow
workflow = StateGraph(FlightBookingState)

# Add nodes
workflow.add_node("parse", parse_travel_request)
workflow.add_node("search", search_flights)
workflow.add_node("analyze", find_cheapest_flight)

# Add edges
workflow.set_entry_point("parse")
workflow.add_conditional_edges(
    "parse",
    should_search_flights,
    {"search": "search", "end": END}
)
workflow.add_conditional_edges(
    "search", 
    should_analyze_flights,
    {"analyze": "analyze", "end": END}
)
workflow.add_edge("analyze", END)

# Compile the graph
flight_booking_agent = workflow.compile()

print("✅ LangGraph workflow created")


✅ LangGraph workflow created


In [45]:
# WhatsApp Bot Function
def process_flight_request(user_message: str) -> str:
    """Process a flight booking request using the LangGraph agent"""
    
    try:
        # Initialize state
        initial_state = FlightBookingState(
            messages=[HumanMessage(content=user_message)],
            user_message=user_message,
            from_city=None,
            to_city=None,
            departure_date=None,
            return_date=None,
            passengers=1,
            passenger_age=25,
            raw_api_response=None,
            cheapest_flight=None,
            response_text=""
        )
        
        # Run the agent
        final_state = flight_booking_agent.invoke(initial_state)
        
        # Return the response
        return final_state.get("response_text", "I'm sorry, I couldn't process your request.")
        
    except Exception as e:
        print(f"❌ Error processing request: {e}")
        return "I'm having trouble processing your request right now. Please try again later."

# Test the bot function
def test_bot():
    """Test the bot with a sample message"""
    test_message = "I want to fly from Lahore to Athens on March 15th and return on March 22nd"
    response = process_flight_request(test_message)
    print(f"Test Input: {test_message}")
    print(f"Bot Response: {response}")

print("✅ Bot processing function ready")


✅ Bot processing function ready


In [46]:
# Flask WhatsApp Webhook Server
app = Flask(__name__)

@app.route("/webhook", methods=["GET", "POST"])
def whatsapp_webhook():
    """Handle WhatsApp webhook requests"""
    
    if request.method == "GET":
        # Webhook verification (for Twilio/Meta setup)
        verify_token = request.args.get("hub.verify_token")
        if verify_token == os.getenv("WEBHOOK_VERIFY_TOKEN", "your_verify_token"):
            return request.args.get("hub.challenge")
        return "Verification failed", 403
    
    elif request.method == "POST":
        # Handle incoming messages
        try:
            # Get message data (format depends on your WhatsApp provider)
            data = request.get_json()
            
            # Extract message text (adjust based on your provider's format)
            message_text = ""
            sender = ""
            
            # For Twilio WhatsApp
            if "Body" in request.form:
                message_text = request.form["Body"]
                sender = request.form["From"]
            
            # For Meta WhatsApp Business API
            elif data and "messages" in data.get("entry", [{}])[0].get("changes", [{}])[0].get("value", {}):
                messages = data["entry"][0]["changes"][0]["value"]["messages"]
                if messages:
                    message_text = messages[0].get("text", {}).get("body", "")
                    sender = messages[0].get("from", "")
            
            if message_text and sender:
                # Process the flight request
                bot_response = process_flight_request(message_text)
                
                # Send response back (format depends on provider)
                return send_whatsapp_response(bot_response, sender)
            
            return "OK", 200
            
        except Exception as e:
            print(f"❌ Webhook error: {e}")
            return "Error", 500
    
    return "Method not allowed", 405

def send_whatsapp_response(message: str, recipient: str):
    """Send response back to WhatsApp user"""
    
    # For Twilio WhatsApp
    resp = MessagingResponse()
    msg = resp.message()
    msg.body(message)
    return str(resp)
    
    # For Meta WhatsApp Business API, you would make a POST request
    # to their Send Message API instead

@app.route("/health")
def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "service": "Flight Booking Bot"}

@app.route("/test", methods=["POST"])
def test_endpoint():
    """Test endpoint for manual testing"""
    data = request.get_json()
    if data and "message" in data:
        response = process_flight_request(data["message"])
        return {"response": response}
    return {"error": "No message provided"}, 400

print("✅ Flask WhatsApp webhook server ready")


✅ Flask WhatsApp webhook server ready


In [47]:
# Environment Variables Setup
environment_setup = """
# Add these to your .env file:
TRAVELPORT_APPLICATION_KEY=your_app_key
TRAVELPORT_APPLICATION_SECRET=your_app_secret  
TRAVELPORT_USERNAME=your_username
TRAVELPORT_PASSWORD=your_password
TRAVELPORT_ACCESS_GROUP=your_access_group
OPENAI_API_KEY=your_openai_key
WEBHOOK_VERIFY_TOKEN=your_webhook_token
"""

# Main execution function
def run_server():
    """Run the Flask server"""
    print("🚀 Starting WhatsApp Flight Booking Bot...")
    print("💡 Make sure to set up your environment variables first!")
    print(environment_setup)
    app.run(host="0.0.0.0", port=5000, debug=True)

print("✅ Setup complete! Ready to run the bot.")


✅ Setup complete! Ready to run the bot.


In [48]:
run_server_fixed()

🚀 Starting WhatsApp Flight Booking Bot...
💡 Make sure to set up your environment variables first!
✅ Flask app configured
🌐 Server will be available at:
   - http://localhost:5000/health
   - http://localhost:5000/webhook
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.18:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Jul/2025 10:25:37] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [18/Jul/2025 10:25:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Jul/2025 10:25:51] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [18/Jul/2025 10:26:17] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [18/Jul/2025 10:26:35] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [18/Jul/2025 10:26:36] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# Quick Test - Run this to test the bot functionality
# Note: Make sure you have all environment variables set up first

def quick_test():
    """Quick test without calling the actual API"""
    print("🧪 Testing the bot workflow (mock mode)...")
    
    # Test message parsing
    test_message = "I want to fly from Lahore to Athens on March 15th 2024"
    
    try:
        # Test the parsing function
        test_state = FlightBookingState(
            messages=[HumanMessage(content=test_message)],
            user_message=test_message,
            from_city=None,
            to_city=None,
            departure_date=None,
            return_date=None,
            passengers=1,
            passenger_age=25,
            raw_api_response=None,
            cheapest_flight=None,
            response_text=""
        )
        
        # Test parsing
        parsed_state = parse_travel_request(test_state)
        print(f"✅ Parsed: From={parsed_state.get('from_city')}, To={parsed_state.get('to_city')}, Date={parsed_state.get('departure_date')}")
        
        print("✅ Bot components are working correctly!")
        print("🚀 Ready to run the full bot with: run_server()")
        
    except Exception as e:
        print(f"❌ Test failed: {e}")
        print("💡 Make sure you have OPENAI_API_KEY set in your environment")

# Uncomment the line below to run the test
# quick_test()
